# GameDay2020 Notebook

The standard dilemma of fantasy baseball is who to choose next: the best available player but in a position you've already filled?; the best at a rare position?; the next best closer because they're going fast?  Whatever choice you make will have consequences, as the next player will no longer have your pick as a choice, and so on.  Ideally you would forecast how each of those would play out and take the best option --- that's exactly what this code does: it identifies the best player left in each of the positions you still need to fill, simulates the rest of draft in each of those scenarios, and returns the best choice.

And it's easy to use!  After [cloning](https://github.com/wrapgenius/GameDay2020) GameDay2020 (and provided you have Jupyter Notebooks with Python 3 installed, and a spreadsheet program) you should have everything you need to do a live roto draft.

The code is based on two ingredients: *projections* and *rankings*:  
- Projections are estimates of player performances for 2020, and are based on data scraped from [Fangraphs](https://www.fangraphs.com/projections.aspx?pos=all&stats=bat&type=zips) (model = ZiPS, Steamer, or TheBat.) 
- Rankings are draft-order recommendations.  The default (i.e.; ranking_method = 'Yahoo') is to use Yahoo Rankings (from Feb 22, 2020).  Others include [ESPN](https://www.espn.com/fantasy/baseball/story/_/id/28285423/fantasy-baseball-2020-rankings-roto-rotisserie), [FantasyPros](https://www.fantasypros.com/mlb/rankings/overall.php) and [Rotographs](https://fantasy.fangraphs.com), or you can put in your own.  Yahoo for yahoo leagues makes most sense since it most closely imitates autodraft.    

_Your_ roto league may have more or less than 12 teams, or use different stats; that ok!  Declare them when defining the Draft object: most of the obvious stats are included, more obscure ones may require hacking into the [fangraphs_projection_2020](https://github.com/wrapgenius/GameDay2020/blob/master/GameDayFunctions/fangraphs_projection_2020.py).  Note, it can't _yet_ do AL or NL only. And adding an injured list is work in progress.  

Below are examples of the different ways the code can be run.  

In [1]:
# Load packages.  Must have pandas and numpy.  

import pdb
import os
import sys
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import copy
from GameDayFunctions.fangraphs_projection_2020 import Projection
from GameDayFunctions.draft_2020 import Draft

In [2]:
year = 2020
path_data = "projections/"
path_drafts = "Draft_Pick_Spreadsheets/"

First need a **Projection** object.  Projections are scraped from [Fangraphs](https://www.fangraphs.com/projections.aspx?pos=all&stats=bat&type=zips), and include ZiPS, Steamer, and TheBat.  
In this example I create a dictionary of the three projections.  

In [6]:
player_ranking_zips = Projection(path_data=path_data,year=year,model='ZiPS',ranking_method = 'Yahoo')
player_ranking_steam = Projection(path_data=path_data,year=year,model='Steamer',ranking_method = 'ESPN')
player_ranking_thebat = Projection(path_data=path_data,year=year,model='TheBat',ranking_method = 'Yahoo')
player_rankings = {"ZiPS":player_ranking_zips, "Steamer":player_ranking_steam, "TheBat":player_ranking_thebat}

Next need a **Draft** object.  The default, e.g.; Draft(player_rankings['ZiPS']), uses 12 teams and the roster spots/roto stats shown in this example. 

In [7]:
# Enter your roto-league-specific stats.  The default settings are shown here.  
draft_position = 2
number_teams = 12
roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':5}
batter_stats  = ['AB','R','1B','2B', '3B','HR','RBI','SB','BB','AVG','OPS']
pitcher_stats = ['IP','W', 'L','CG','SHO','SV','BB','SO','ERA','WHIP','BSV'] 

# Choose the projection system you prefer ()
player_projections = player_rankings['ZiPS']

# Get an instance of the Draft Class with your league-specific details and projection preference.
zips_example = Draft(player_projections, 
                     draft_position = draft_position, 
                     number_teams = number_teams,
                     roster_spots = roster_spots,
                     batter_stats = batter_stats,
                     pitcher_stats = pitcher_stats)

To use while drafting in real-time, simply record the draft results in a spreadsheet (.xlsx, so if you're using Numbers, need to export to Excel) and run the cell. 

Examples below show results for after 2, 3, and 4 rounds.  Notice that at each stage the projected placement in the standings improves.  

In [ ]:
# Example pick in the 2nd round

zipsT = Draft(player_rankings['ZiPS'], draft_position = 5)
player_to_pick, projected_roto_stats = zipsT.draft_from_list_and_find_best_pick(draft_pick_file = 'TestPicksround2.xlsx', path_list = path_drafts)

Team 1 picking Mike Trout for OF
Team 2 picking Mookie Betts for OF
Team 3 picking Alex Bregman for SS
Team 4 picking Anthony Rendon for 3B
Team 5 picking Nolan Arenado for 3B
Team 6 picking Jacob deGrom for SP
Team 7 picking Christian Yelich for OF
Team 8 picking Juan Soto for OF
Team 9 picking Cody Bellinger for OF
Team 10 picking Ronald Acuna for OF
Team 11 picking Justin Verlander for SP
Team 12 picking Francisco Lindor for SS
Team 12 picking Jose Altuve for 2B
Team 11 picking Max Scherzer for SP
Team 10 picking Fernando Tatis for SS
Team 9 picking Trevor Story for SS
Team 8 picking Charlie Blackmon for OF
Team 7 picking Whit Merrifield for 2B
Team 6 picking Freddie Freeman for 1B
Finding Best Pick For Team 5
Stored Result for Pick 0 J.D. Martinez OF whose placing/score is [9, 105.0]
Stored Result for Pick 1 Xander Bogaerts SS whose placing/score is [10, 133.5]


In [6]:
# Example pick in the 3rd round

zipsT = Draft(player_rankings['ZiPS'], draft_position = 5)
player_to_pick, projected_roto_stats = zipsT.draft_from_list_and_find_best_pick(draft_pick_file = 'TestPicksround3.xlsx', path_list = path_drafts)

Team 1 picking Mike Trout for OF
Team 2 picking Mookie Betts for OF
Team 3 picking Alex Bregman for SS
Team 4 picking Anthony Rendon for 3B
Team 5 picking Nolan Arenado for 3B
Team 6 picking Jacob deGrom for SP
Team 7 picking Christian Yelich for OF
Team 8 picking Juan Soto for OF
Team 9 picking Cody Bellinger for OF
Team 10 picking Ronald Acuna for OF
Team 11 picking Justin Verlander for SP
Team 12 picking Francisco Lindor for SS
Team 12 picking Jose Altuve for 2B
Team 11 picking Max Scherzer for SP
Team 10 picking Fernando Tatis for SS
Team 9 picking Trevor Story for SS
Team 8 picking Charlie Blackmon for OF
Team 7 picking Whit Merrifield for 2B
Team 6 picking Freddie Freeman for 1B
Team 5 picking Trea Turner for SS
Team 4 picking Bryce Harper for OF
Team 3 picking Chris Sale for SP
Team 2 picking Walker Buehler for SP
Team 1 picking Shane Bieber for SP
Team 1 picking Stephen Strasburg for SP
Team 2 picking Josh Hader for RP
Team 3 picking Rafael Devers for 3B
Team 4 picking Ketel Ma

In [7]:
# Example pick in the 4th round

zipsT = Draft(player_rankings['ZiPS'], draft_position = 5)
player_to_pick, projected_roto_stats = zipsT.draft_from_list_and_find_best_pick(draft_pick_file = 'TestPicksround4.xlsx', path_list = path_drafts)

Team 1 picking Mike Trout for OF
Team 2 picking Mookie Betts for OF
Team 3 picking Alex Bregman for SS
Team 4 picking Anthony Rendon for 3B
Team 5 picking Nolan Arenado for 3B
Team 6 picking Jacob deGrom for SP
Team 7 picking Christian Yelich for OF
Team 8 picking Gerrit Cole for SP
Team 9 picking Cody Bellinger for OF
Team 10 picking Juan Soto for OF
Team 11 picking Justin Verlander for SP
Team 12 picking Francisco Lindor for SS
Team 12 picking Jose Altuve for 2B
Team 11 picking Max Scherzer for SP
Team 10 picking Ronald Acuna for OF
Team 9 picking Trevor Story for SS
Team 8 picking Charlie Blackmon for OF
Team 7 picking Whit Merrifield for 2B
Team 6 picking Freddie Freeman for 1B
Team 5 picking Trea Turner for SS
Team 4 picking Bryce Harper for OF
Team 3 picking Chris Sale for SP
Team 2 picking Walker Buehler for SP
Team 1 picking Shane Bieber for SP
Team 1 picking Stephen Strasburg for SP
Team 2 picking Josh Hader for RP
Team 3 picking Rafael Devers for 3B
Team 4 picking Fernando Ta

In [8]:
# Projected_roto_stats contains the estimated results from all teams given the recommented pick

projected_stats = projected_roto_stats[0]
projected_standings = projected_roto_stats[3]
projected_place = projected_roto_stats[4]
projected_scores = projected_roto_stats[5]

In [9]:
# The projected_stats give the raw numbers

projected_stats

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BB,SO,ERA,WHIP,BSV
0,932.0,957.0,326.0,28.0,331.0,1025.0,92.0,624.0,0.262143,0.813784,72.0,43.0,5.0,5.0,17.0,286.0,1169.0,3.539858,1.140051,1.0
1,949.0,962.0,336.0,31.0,274.0,918.0,106.0,596.0,0.264531,0.801797,67.0,43.0,5.0,5.0,38.0,294.0,1196.0,3.488056,1.135566,3.0
2,1204.0,1217.0,425.0,48.0,378.0,1234.0,142.0,722.0,0.268971,0.819091,63.0,43.0,4.0,4.0,53.0,254.0,901.0,3.808359,1.206610,4.0
3,850.0,841.0,304.0,22.0,293.0,970.0,46.0,618.0,0.266481,0.833953,60.0,46.0,5.0,5.0,0.0,321.0,1030.0,3.972525,1.232302,0.0
4,1047.0,1109.0,358.0,46.0,321.0,1065.0,156.0,663.0,0.269049,0.813548,56.0,38.0,4.0,4.0,29.0,246.0,858.0,3.991989,1.221219,2.0
5,895.0,945.0,332.0,36.0,283.0,989.0,111.0,638.0,0.261286,0.804911,80.0,50.0,4.0,4.0,53.0,332.0,1318.0,3.638573,1.162609,4.0
6,1019.0,1152.0,384.0,47.0,312.0,1125.0,150.0,671.0,0.266098,0.798321,51.0,38.0,3.0,3.0,81.0,235.0,792.0,3.789221,1.208440,7.0
7,824.0,903.0,278.0,39.0,209.0,722.0,95.0,591.0,0.269444,0.801471,89.0,56.0,6.0,6.0,51.0,385.0,1477.0,3.676812,1.182724,5.0
8,1121.0,1175.0,403.0,44.0,345.0,1119.0,166.0,752.0,0.270178,0.821774,65.0,48.0,5.0,5.0,24.0,319.0,977.0,3.880004,1.267141,2.0
9,812.0,864.0,302.0,33.0,258.0,884.0,63.0,583.0,0.273727,0.836807,84.0,60.0,7.0,7.0,25.0,381.0,1286.0,3.669872,1.207725,2.0


In [10]:
# And projected_scores show how they translate to roto scores

projected_scores

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BBP,SO,ERA,WHIP,BSV
0,6.0,6.0,5.0,2.0,10.0,8.0,3.0,5.0,3.0,7.0,8.0,8.0,8.0,8.0,2.0,8.0,7.0,11.0,11.0,11.0
1,7.0,7.0,7.0,3.5,4.0,4.0,5.0,3.0,4.0,3.0,7.0,8.0,8.0,8.0,7.0,7.0,8.0,12.0,12.0,6.0
2,12.0,12.0,12.0,11.5,12.0,12.0,8.0,10.0,8.0,8.0,5.0,8.0,4.0,4.0,9.5,10.0,4.0,5.0,7.0,4.5
3,3.0,1.0,4.0,1.0,6.0,5.0,1.0,4.0,7.0,11.0,4.0,6.0,8.0,8.0,1.0,4.0,6.0,2.0,2.0,12.0
4,10.0,9.0,9.0,9.0,8.0,9.0,10.0,8.0,9.0,6.0,3.0,11.0,4.0,4.0,6.0,11.0,3.0,1.0,3.0,8.5
5,5.0,5.0,6.0,6.0,5.0,6.0,6.0,7.0,2.0,4.0,9.5,4.0,4.0,4.0,9.5,3.0,11.0,10.0,9.0,4.5
6,8.0,10.0,10.0,10.0,7.0,11.0,9.0,9.0,6.0,1.0,1.0,11.0,1.5,1.5,12.0,12.0,1.0,6.0,5.0,1.0
7,2.0,4.0,1.0,7.0,1.0,1.0,4.0,2.0,10.0,2.0,12.0,2.0,11.0,11.0,8.0,1.0,12.0,7.0,8.0,2.5
8,11.0,11.0,11.0,8.0,11.0,10.0,12.0,12.0,11.0,9.0,6.0,5.0,8.0,8.0,4.0,5.0,5.0,4.0,1.0,8.5
9,1.0,2.0,2.5,5.0,2.0,3.0,2.0,1.0,12.0,12.0,11.0,1.0,12.0,12.0,5.0,2.0,10.0,9.0,6.0,8.5


In [11]:
# Finally, the projected_standings shows the resulting standings

projected_standings

2     166.5
8     160.5
4     141.5
11    137.0
0     137.0
6     133.0
1     130.5
5     120.5
9     119.0
10    110.0
7     108.5
3      96.0
dtype: float64

In [12]:
# But if you just want to know how your team finishes, that's projected_place

projected_place

3

### You can simulate drafts as well. Naive drafts simply picks the next eligible (i.e., open roster spot) player on the ranked list. 

In [13]:
# Try a quick draft: 4 teams, limited roster spots, naive (i.e., next available pick on ranked list) picks.

zipsS= Draft(player_rankings['ZiPS'], draft_position = 2, number_teams = 4, roster_spots = {'C':0,'1B':0,'2B':1, '3B':1,'SS':1,'OF':2,'UTIL':1,'SP':1,'RP':1,'P':1,'BN':1})
zipsS.draft_all(naive_draft = True)
print('Placement = '+str(zipsS.roto_placement))

Placement = 3


In [14]:
# Now try a naive draft again, but for full 12 team and all 24 roster spots.  Draft Position is 2 

zipsNaive_2 = Draft(player_rankings['ZiPS'], draft_position = 2)
zipsNaive_2.draft_all(naive_draft = True)
print('Placement  = '+str(zipsNaive_2.roto_placement))

Placement  = 9


In [15]:
# This time do the full-simulation, recommended picks version, i.e.; naive_draft = False
# Remember, it will try every remaining open position and pick the one whose resulting placement is highest.  

zips_2 = Draft(player_rankings['ZiPS'], draft_position = 2)
zips_2.draft_all(naive_draft = False)
print('Placement  = '+str(zips_2.roto_placement))

Team 2 picking J.T. Realmuto for C
Team 2 picking Jose Altuve for 2B
Team 2 picking Manny Machado for 3B
Team 2 picking Paul Goldschmidt for 1B
Team 2 picking Andrew Benintendi for OF
Team 2 picking Adalberto Mondesi for SS
Team 2 picking Noah Syndergaard for SP
Team 2 picking Aroldis Chapman for RP
Team 2 picking Corey Seager for UTIL
Team 2 picking Raisel Iglesias for OF
Team 2 picking Luis Robert for BN
Team 2 picking Trey Mancini for OF
Team 2 picking Zac Gallen for SP
Team 2 picking Elvis Andrus for BN
Team 2 picking Kyle Tucker for BN
Team 2 picking Michael Chavis for BN
Team 2 picking Ryan McMahon for BN
Team 2 picking Seth Lugo for RP
Team 2 picking Mike Foltynewicz for P
Team 2 picking James Paxton for P
Team 2 picking Jose Urquidy for P
Placement  = 1


In [16]:
# Lets compare Naive True/False for the 7th pick

zipsNaive_7 = Draft(player_rankings['ZiPS'], draft_position = 7)
zipsNaive_7.draft_all(naive_draft = True)
print('Placement  = '+str(zipsNaive_7.roto_placement))

Placement  = 12


In [18]:
zips_7 = Draft(player_rankings['ZiPS'], draft_position = 7)
zips_7.draft_all(naive_draft = False)
print('Placement  = '+str(zips_7.roto_placement))

Team 7 picking J.T. Realmuto for C
Team 7 picking DJ LeMahieu for 1B
Team 7 picking Chris Sale for SP
Team 7 picking Keston Hiura for 2B
Team 7 picking Carlos Santana for RP
Team 7 picking Josh Bell for UTIL
Team 7 picking Hunter Dozier for OF
Team 7 picking Chris Paddack for SP
Team 7 picking Tommy Edman for 3B
Team 7 picking Luis Severino for P
Team 7 picking Victor Robles for OF
Team 7 picking Aaron Bummer for RP
Team 7 picking Matt Olson for BN
Team 7 picking Eddie Rosario for OF
Team 7 picking Jean Segura for SS
Team 7 picking Tyler Rogers for P
Team 7 picking Andrew Chafin for BN
Team 7 picking Ryan Yarbrough for P
Team 7 picking Julio Urías for BN
Team 7 picking Ty Buttrey for BN
Team 7 picking Cam Bedrosian for BN
Placement  = 1


In [111]:
# And for the 11th pick

zipsNaive_11 = Draft(player_rankings['ZiPS'], draft_position = 11)
zipsNaive_11.draft_all(naive_draft = True)
print('Placement  = '+str(zipsNaive_11.roto_placement))

Placement  = 4


In [33]:
zips_11 = Draft(player_rankings['ZiPS'], draft_position = 11)
zips_11.draft_all(naive_draft = False)
print('Placement  = '+str(zips_11.roto_placement))

Team 11 picking Rafael Devers for 3B
Team 11 picking Charlie Blackmon for OF
Team 11 picking Keston Hiura for 2B
Team 11 picking Gerrit Cole for P
Team 11 picking Nick Castellanos for OF
Team 11 picking Josh Bell for SS
Team 11 picking Adam Ottavino for RP
Team 11 picking Manny Machado for UTIL
Team 11 picking Tommy Edman for OF
Team 11 picking Adam Kolarek for RP
Team 11 picking Yu Darvish for SP
Team 11 picking Mike Soroka for SP
Team 11 picking J.T. Realmuto for C
Team 11 picking Elvis Andrus for BN
Team 11 picking Richard Rodríguez for P
Team 11 picking Tim Anderson for BN
Team 11 picking Michael Lorenzen for P
Team 11 picking Sam Tuivailala for BN
Team 11 picking Shohei Ohtani for BN
Team 11 picking Mychal Givens for BN
Team 11 picking Ryan McMahon for 1B
Placement  = 1


In [31]:
# We can do simlated drafts for different projection systems.  Above was all with ZiPs.  Lets try a Steamer, with custom league settings.

number_teams = 6
roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':2}
steam_custom = Draft(player_rankings['ZiPS'], number_teams=number_teams, roster_spots=roster_spots)
steam_custom.draft_all(naive_draft = True)
print('Placement  = '+str(steam_custom.roto_placement))

Placement  = 5


In [32]:
# And compare Naive to the recommendations from the simulation

roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':2}
steam_custom = Draft(player_rankings['ZiPS'], number_teams=number_teams, roster_spots=roster_spots)
steam_custom.draft_all(naive_draft = False)
print('Placement  = '+str(steam_custom.roto_placement))

Team 2 picking J.T. Realmuto for C
Team 2 picking Trevor Story for BN
Swapping UTIL with OF
Team 2 picking Charlie Blackmon for OF
Team 2 picking Trea Turner for SS
Team 2 picking Ketel Marte for 2B
Team 2 picking DJ LeMahieu for 1B
Team 2 picking Chris Sale for SP
Team 2 picking Carlos Santana for UTIL
Team 2 picking Tyler Glasnow for SP
Team 2 picking Paul Goldschmidt for BN
Team 2 picking Sean Doolittle for RP
Team 2 picking Zack Greinke for P
Team 2 picking Starling Marte for OF
Team 2 picking Ken Giles for RP
Team 2 picking Andrew Benintendi for OF
Team 2 picking Ian Kennedy for P
Team 2 picking Ryan Pressly for P
Team 2 picking Tommy Edman for 3B
Placement  = 1


In [20]:
# And lets compare Naive True/False for different draft positions

steam_2= Draft(player_rankings['Steamer'], draft_position = 2)
steam_2.draft_all(naive_draft = True)
print('Placement  = '+str(steam_2.roto_placement))

Placement  = 10


In [21]:
steam_2= Draft(player_rankings['Steamer'], draft_position = 2)
steam_2.draft_all(naive_draft = False)
print('Placement  = '+str(steam_2.roto_placement))

Team 2 picking J.T. Realmuto for C
Team 2 picking Ketel Marte for SP
Team 2 picking Rafael Devers for 3B
Team 2 picking Mike Clevinger for SP
Swapping UTIL with OF
Team 2 picking George Springer for OF
Team 2 picking Anthony Rizzo for SS
Team 2 picking Starling Marte for OF
Team 2 picking Josh Donaldson for UTIL
Team 2 picking Chris Paddack for P
Team 2 picking Tommy Kahnle for BN
Team 2 picking Matt Chapman for BN
Team 2 picking Gleyber Torres for 2B
Team 2 picking Yonny Chirinos for RP
Team 2 picking Yuli Gurriel for 1B
Team 2 picking Andrew McCutchen for OF
Team 2 picking Matt Manning for P
Team 2 picking Dellin Betances for P
Team 2 picking Carl Edwards for RP
Team 2 picking Forrest Whitley for BN
Team 2 picking Ty Buttrey for BN
Team 2 picking Cam Bedrosian for BN
Placement  = 1


In [22]:
steam_7= Draft(player_rankings['Steamer'], draft_position = 7)
steam_7.draft_all(naive_draft = True)
print('Placement  = '+str(steam_7.roto_placement))

Placement  = 12


In [23]:
steam_7= Draft(player_rankings['Steamer'], draft_position = 7)
steam_7.draft_all(naive_draft = False)
print('Placement  = '+str(steam_7.roto_placement))

Team 7 picking J.T. Realmuto for C
Team 7 picking DJ LeMahieu for 1B
Team 7 picking Josh Hader for RP
Swapping UTIL with OF
Team 7 picking Aaron Judge for OF
Team 7 picking Paul Goldschmidt for BN
Team 7 picking Michael Conforto for OF
Team 7 picking Jeff McNeil for 2B
Team 7 picking Chris Paddack for SP
Swapping UTIL with OF
Team 7 picking Trey Mancini for OF
Team 7 picking Luis Severino for SP
Team 7 picking Colin Poche for RP
Team 7 picking Yoan López for BN
Team 7 picking Matt Olson for UTIL
Team 7 picking Eddie Rosario for BN
Team 7 picking Emmanuel Clase for P
Team 7 picking Carlos Correa for SS
Team 7 picking Seranthony Domínguez for P
Team 7 picking Carlos Carrasco for P
Team 7 picking David Fletcher for 3B
Team 7 picking Marcus Walden for BN
Team 7 picking Ty Buttrey for BN
Placement  = 1


In [24]:
steam_11= Draft(player_rankings['Steamer'], draft_position = 11)
steam_11.draft_all(naive_draft = True)
print('Placement  = '+str(steam_11.roto_placement))

Placement  = 4


In [114]:
steam_11= Draft(player_rankings['Steamer'], draft_position = 11)
steam_11.draft_all(naive_draft = False)
print('Placement  = '+str(steam_11.roto_placement))

Team 10 picking Rafael Devers for 3B
Team 10 picking Jacob deGrom for SP
Team 10 picking J.T. Realmuto for C
Team 10 picking Chris Sale for SP
Team 10 picking Jeff McNeil for 2B
Team 10 picking Jorge Polanco for SS
Team 10 picking Andrew Benintendi for OF
Team 10 picking Hunter Dozier for 1B
Team 10 picking Pedro Báez for RP
Swapping UTIL with OF
Team 10 picking Trey Mancini for OF
Team 10 picking Andres Muñoz for RP
Team 10 picking Mike Soroka for BN
Team 10 picking Elvis Andrus for UTIL
Team 10 picking Eddie Rosario for OF
Team 10 picking Jean Segura for BN
Team 10 picking Tim Anderson for BN
Team 10 picking Oliver Drake for P
Team 10 picking Sam Tuivailala for P
Team 10 picking Ty Buttrey for P
Team 10 picking Cam Bedrosian for BN
Team 10 picking Mychal Givens for BN
Placement  = 1


In [19]:
# And finally, 7th pick using TheBat projection system. 

thebat_7= Draft(player_rankings['TheBat'], draft_position = 7)
thebat_7.draft_all(naive_draft = True)
print('Placement  = '+str(thebat_7.roto_placement))

Placement  = 12


In [20]:
thebat_7= Draft(player_rankings['TheBat'], draft_position = 7)
thebat_7.draft_all(naive_draft = False)
print('Placement  = '+str(thebat_7.roto_placement))

Team 7 picking J.T. Realmuto for C
Team 7 picking Xander Bogaerts for BN
Team 7 picking Chris Sale for SP
Team 7 picking Tommy Pham for OF
Swapping UTIL with 1B
Team 7 picking Paul Goldschmidt for 1B
Team 7 picking Josh Bell for UTIL
Team 7 picking Chris Paddack for SP
Team 7 picking Michael Brantley for OF
Team 7 picking Scott Oberg for RP
Team 7 picking Luis Severino for P
Team 7 picking Victor Robles for OF
Team 7 picking Aaron Bummer for RP
Team 7 picking Matt Olson for BN
Team 7 picking Eddie Rosario for BN
Team 7 picking Jean Segura for SS
Team 7 picking Tyler Rogers for P
Team 7 picking Kyle Crick for BN
Team 7 picking Carlos Carrasco for P
Team 7 picking Ty Buttrey for BN
Team 7 picking Gavin Lux for 2B
Team 7 picking David Fletcher for 3B
Placement  = 1


In [256]:
# If you want to see the picks happening in real-time, choose silent=False

zips = Draft(player_rankings['ZiPS'])
zips.draft_all(silent=False)

Picking from:
                PLAYER EligiblePosition
Rank                                   
2         Mookie Betts               OF
3           Mike Trout               OF
4         Ronald Acuña               OF
6       Anthony Rendon               3B
7       Cody Bellinger            1B/OF
9         Alex Bregman            3B/SS
11    Justin Verlander               SP
13        Jacob deGrom               SP
15     Whit Merrifield            2B/OF
27       Roberto Osuna               RP
185      J.T. Realmuto                C
Stored Result for Pick 0 Mookie Betts OF whose score is [8, 177.5]
Stored Result for Pick 1 Mike Trout UTIL whose score is [5, 132.5]
Stored Result for Pick 2 Ronald Acuña BN whose score is [9, 119.5]
Stored Result for Pick 3 Anthony Rendon 3B whose score is [6, 129.5]
Stored Result for Pick 4 Cody Bellinger 1B whose score is [6, 139.0]
Stored Result for Pick 5 Alex Bregman SS whose score is [6, 131.0]
Stored Result for Pick 6 Justin Verlander SP whose score is 